This script process DNA and RNA  integration results and merge them together side by side. It is written for Cervical project.

In [157]:
import matplotlib
import seaborn
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import scipy.stats as ss
from matplotlib import style
matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (15, 9)
import pandas as pd
# figure out number of indels
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

This function process groupby object, combine all rows with the same patient/virus indexed entries into one row

In [288]:
def process_grp(grp):
    df_or_sr = grp.apply(lambda col: 
                          ';'.join(col.unique().tolist()) 
                          if len(col.unique())==1 
                          else ';'.join(col.tolist()))
    df = df_or_sr.to_frame().T
    assert df.shape[0] == 1
    return df

# files

In [ ]:
# cat /projects/NCI_validation2_assembly/NCI_SAIC_HIV_Cervical/integration_sites/NCI_SAIC_HIV_Cervical/genome/integration_results/HTMCP*/integration_summary_viral_annotated.tsv|grep -v viral|grep -v No-Hits

In [289]:
wkdir = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/'
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/genomes_with_htmcp_ids_118_20191217.tsv'
dna_f = f'{wkdir}DNA_integration_summary_20191217.tsv' # concate all integration_summary_viral_annotated.tsv files
rna_f = f'{wkdir}RNA_integration_summary_20191217.tsv'
dna_f

'/projects/trans_scratch/validations/workspace/szong/Cervical/integration/DNA_integration_summary_20191217.tsv'

In [290]:
df = pd.read_csv(of, sep='\t')
df.head(2)
df.shape

,Tissue Site,HIV status,GSC original source ID,patient,DNA_lib,status,RNA_lib
0,Cervix,positive,HTMCP_124,HTMCP-03-06-02001,A37234,Malignant,A37700
1,Cervix,Negative,HTMCP_125,HTMCP-03-06-02002,A37235,Malignant,A37701


(118, 7)

# genome

In [302]:
dfg = pd.read_csv(dna_f, sep='\t')
dfg.head(2)
dfg.shape
dfgm = pd.merge(dfg, df, left_on='library', right_on='GSC original source ID', how='left')
dfgm.drop('GSC original source ID', axis=1, inplace=True)
dfgm.head(2)
dfgm.shape

dfgm['virus'] = dfgm['virus'].str.replace('Human papillomavirus type ', 'HPV_')
dfgf = dfgm[dfgm.Integration == 'YES'].groupby(['patient', 'virus']).apply(process_grp).reset_index(level=2, drop=True)
dfgf = dfgf[['viral-breakpoint', 'viral-gene', 'DNA_lib', 'RNA_lib']].reset_index()
# dfgf = dfgf
dfgf.shape
dfgf.head(2)

,library,virus,Integration,human-breakpoint,human-gene(s),viral-breakpoint,viral-gene
0,HTMCP_126,Human papillomavirus type 16,NO,-,-,-,-
1,HTMCP_133,Human papillomavirus type 73,YES,17_36310359,intergenic,5956,-


(368, 7)

,library,virus,Integration,human-breakpoint,human-gene(s),viral-breakpoint,viral-gene,Tissue Site,HIV status,patient,DNA_lib,status,RNA_lib
0,HTMCP_126,Human papillomavirus type 16,NO,-,-,-,-,Cervix,positive,HTMCP-03-06-02003,A37236,Malignant,A37702
1,HTMCP_133,Human papillomavirus type 73,YES,17_36310359,intergenic,5956,-,Cervix,Negative,HTMCP-03-06-02012,A37243,Malignant,A37707


(368, 13)

(60, 6)

,patient,virus,viral-breakpoint,viral-gene,DNA_lib,RNA_lib
0,HTMCP-03-06-02012,HPV_73,5956,-,A37243,A37707
1,HTMCP-03-06-02042,HPV_16,2237;4267,E1;L2,A37255,A37717


# RNA

In [356]:
df.head(2)

,Tissue Site,HIV status,GSC original source ID,patient,DNA_lib,status,RNA_lib
0,Cervix,positive,HTMCP_124,HTMCP-03-06-02001,A37234,Malignant,A37700
1,Cervix,Negative,HTMCP_125,HTMCP-03-06-02002,A37235,Malignant,A37701


In [365]:
df[df.patient == 'HTMCP-03-06-02204']

,Tissue Site,HIV status,GSC original source ID,patient,DNA_lib,status,RNA_lib
76,Cervix,Positive,HTMCP_401,HTMCP-03-06-02204,A57086,Malignant,A57113


In [357]:
dfr = pd.read_csv(rna_f, sep='\t')
dfr.head(2)
dfr.shape
dfrm = pd.merge(dfr, df, left_on='library', right_on='RNA_lib', how='left')
print('number of patients', dfrm.patient.nunique())
dfrm.head(2)
dfrm.shape

dfrm['virus'] = dfrm['virus'].str.replace('Human papillomavirus type ', 'HPV_')
dfrm['viral-gene'] = dfrm['viral-gene'].fillna('NA')
dfrf = dfrm[dfrm.Integration == 'YES'].groupby(['patient', 'virus']).apply(process_grp).reset_index(level=2, drop=True)
dfrf = dfrf[['viral-breakpoint', 'viral-gene', 'DNA_lib', 'RNA_lib']].reset_index()
print('number of patients', dfrf.patient.nunique())
dfrf.shape
dfrf.head(2)

,library,virus,Integration,human-breakpoint,human-gene(s),viral-breakpoint,viral-gene
0,A37700,Human papillomavirus type 16,YES,3_189526056,TP63,880,E1
1,A37700,Human papillomavirus type 16,YES,3_189526059,TP63,3631,E2


(951, 7)

number of patients 118


,library,virus,Integration,human-breakpoint,human-gene(s),viral-breakpoint,viral-gene,Tissue Site,HIV status,GSC original source ID,patient,DNA_lib,status,RNA_lib
0,A37700,Human papillomavirus type 16,YES,3_189526056,TP63,880,E1,Cervix,positive,HTMCP_124,HTMCP-03-06-02001,A37234,Malignant,A37700
1,A37700,Human papillomavirus type 16,YES,3_189526059,TP63,3631,E2,Cervix,positive,HTMCP_124,HTMCP-03-06-02001,A37234,Malignant,A37700


(951, 14)

number of patients 101


(110, 6)

,patient,virus,viral-breakpoint,viral-gene,DNA_lib,RNA_lib
0,HTMCP-03-06-02001,HPV_16,880;3631;5695;408;3631;880;3689;3753,E1;E2;L1;E6;E2;E1;E2;E2,A37234,A37700
1,HTMCP-03-06-02002,HPV_52,1307;880;1723;881;879,-,A37235,A37701


In [358]:
dfrm['viral-gene'].unique()

array(['E1', 'E2', 'L1', 'E6', '-', 'L2', 'E4,E2', 'URR', 'E2,E4', 'E7',
       '41-16', 'E5_ALPHA', 'NA', 'E1,E2', 'E2,E1'], dtype=object)

In [359]:
dfrf[dfrf.patient=='HTMCP-03-06-02097']

,patient,virus,viral-breakpoint,viral-gene,DNA_lib,RNA_lib
27,HTMCP-03-06-02097,HPV_16,2077;5178,E1;L2,A37283,A37736
28,HTMCP-03-06-02097,HPV_59,3269;888;889;889;745;887;743,-,A37283,A37736


In [360]:
dfrf['virus'].unique()

array(['HPV_16', 'HPV_52', 'HPV_45', 'HPV_9', 'HPV_73', 'HPV_18',
       'HPV_58', 'HPV_59', 'HPV_31', 'HPV_26', 'HPV_33', 'HPV_82',
       'HPV_97', 'HPV_69', 'HPV_35', 'HPV_68b', 'HPV_68a', 'HPV_30'],
      dtype=object)

# merge DNA and RNA results

In [363]:
dfm = pd.merge(dfgf, dfrf, on=['patient', 'virus'], how='outer', suffixes=('_DNA', '_RNA'))
dfm.head()
dfm = dfm.fillna('na')
dfm.shape

,patient,virus,viral-breakpoint_DNA,viral-gene_DNA,DNA_lib_DNA,RNA_lib_DNA,viral-breakpoint_RNA,viral-gene_RNA,DNA_lib_RNA,RNA_lib_RNA
0,HTMCP-03-06-02012,HPV_73,5956,-,A37243,A37707,863;157;1959;1959,-,A37243,A37707
1,HTMCP-03-06-02042,HPV_16,2237;4267,E1;L2,A37255,A37717,880;226;886;881;2237;4325;881;880;3419;881;880...,"E1;E6;E1;E1;E1;L2;E1;E1;E4,E2;E1;E1;E6;E1;E1",A37255,A37717
2,HTMCP-03-06-02046,HPV_16,2895;683;834,E2;E7;E7,A37257,A37718,2895;6539;880;7394;3356;3420;2644;881;3416;335...,"E2;L1;E1;URR;E2,E4;E2,E4;E1;E1;E2,E4;E2,E4;E7;...",A37257,A37718
3,HTMCP-03-06-02058,HPV_18,2232;7739;7528,E1;URR;URR,A37261,A37722,6945;778;235;931;1359;931;7739;929;7528,L1;E7;E6;E1;E1;E1;URR;E1;URR,A37261,A37722
4,HTMCP-03-06-02063,HPV_16,2692;1774;7642,E1;E1;URR,A37266,A37725,880;1774;880;1301;226;880;880;3419;1260,"E1;E1;E1;E1;E6;E1;E1;E4,E2;E1",A37266,A37725


(124, 10)

In [364]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/118_patient_integration_viral_20191218.tsv'
# dfm.to_csv(of, sep='\t', index=False)

In [415]:
of2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/118_patient_integration_viral_fixed_20191218.tsv'
dfm = pd.read_csv(of2, sep='\t')
dfm.shape
dfm.head(2)

(117, 8)

,patient,virus,viral-breakpoint_DNA,viral-gene_DNA,DNA_lib,RNA_lib,viral-breakpoint_RNA,viral-gene_RNA
0,HTMCP-03-06-02204,HPV_18,na,na,A57086,A57113,1735,E1
1,HTMCP-03-06-02012,HPV_73,5956,-,A37243,A37707,863;157;1959;1959,-


In [416]:
dfm.shape

(117, 8)

In [417]:
dfm[dfm.virus == 'HPV_16'].shape
dfm[dfm.virus == 'HPV_18'].shape

(40, 8)

(29, 8)

In [418]:
dfm.patient.nunique()

111

In [419]:
f5 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/118_patients_integration_details_merged_comparison_to_review_apv_removed_20190729_excel_cleaned.xls'
df5 = pd.read_excel(f5, sep='\t')
df5.head()
df5.shape

,patient,virus,comments,integration_status,should_exclude_due_to_review,DNA_human-breakpoint,DNA_human-gene(s),RNA_human-breakpoint,RNA_human-gene(s)
0,HTMCP-03-06-02068,HPV_59,false_positive,not_detected,YES,na,na,1_209560518,intergenic
1,HTMCP-03-06-02098,HPV_9,false_positive,not_detected,YES,2_137648071,THSD7B,na,na
2,HTMCP-03-06-02110,HPV_45,false_positive,not_detected,YES,8_128302005,"DQ515899,DQ515898,LOC727677",na,na
3,HTMCP-03-06-02176,HPV_58,false_positive,not_detected,YES,4_140811717,MAML3,4_140811717,MAML3
4,HTMCP-03-06-02006,HPV_18,hpv18_false_positive,not_detected,YES,na,na,-,-


(179, 9)

In [420]:
dfi = df5[df5.integration_status=='integrated']
dfi.shape
dfi.integration_status.unique()
pat1 = dfi.patient.unique()
len(pat1)
virus1 = dfi.virus.unique()
len(virus1)

(113, 9)

array(['integrated'], dtype=object)

110

16

In [421]:
pat2 = dfm.patient.unique()
len(pat2)
virus2 = dfm.virus.unique()
len(virus2)

111

17

In [422]:
dfm.head()

,patient,virus,viral-breakpoint_DNA,viral-gene_DNA,DNA_lib,RNA_lib,viral-breakpoint_RNA,viral-gene_RNA
0,HTMCP-03-06-02204,HPV_18,na,na,A57086,A57113,1735,E1
1,HTMCP-03-06-02012,HPV_73,5956,-,A37243,A37707,863;157;1959;1959,-
2,HTMCP-03-06-02042,HPV_16,2237;4267,E1;L2,A37255,A37717,880;226;886;881;2237;4325;881;880;3419;881;880...,"E1;E6;E1;E1;E1;L2;E1;E1;E4,E2;E1;E1;E6;E1;E1"
3,HTMCP-03-06-02046,HPV_16,2895;683;834,E2;E7;E7,A37257,A37718,2895;6539;880;7394;3356;3420;2644;881;3416;335...,"E2;L1;E1;URR;E2,E4;E2,E4;E1;E1;E2,E4;E2,E4;E7;..."
4,HTMCP-03-06-02058,HPV_18,2232;7739;7528,E1;URR;URR,A37261,A37722,6945;778;235;931;1359;931;7739;929;7528,L1;E7;E6;E1;E1;E1;URR;E1;URR


In [423]:
set(pat1) - set(pat2)
set(pat2) - set(pat1)

set()

{'HTMCP-03-06-02217'}

In [424]:
set(virus1) - set(virus2)
set(virus2) - set(virus1)

{'HPV_68'}

{'HPV_68a', 'HPV_68b'}

In [406]:
# removed these three lines from the dfm to reflect review findings:
# HTMCP-03-06-02179	HPV_97	7757	-	A54381	A54473	na	na
# HTMCP-03-06-02259	HPV_97	7701	-	A68233	A68252	na	na
# HTMCP-03-06-02219	HPV_97	na	na	A57089	A57116	3579	NA


HTMCP-03-06-02204 determined integrated by reviewing, so understandable does not report viral breakpoint by pipeline, HPV18 integration was found in HTMCP-03-06-02204 HPV18 breakpoint appears to be near 1735, E1 gene.

In [349]:
for p in ['HTMCP-03-06-02204','HTMCP-03-06-02217']:
    dfm[dfm['patient'] == p]
    dfi[dfi.patient== p]
    

,patient,virus,viral-breakpoint_DNA,viral-gene_DNA,DNA_lib_DNA,RNA_lib_DNA,viral-breakpoint_RNA,viral-gene_RNA,DNA_lib_RNA,RNA_lib_RNA


,patient,virus,comments,integration_status,should_exclude_due_to_review,DNA_human-breakpoint,DNA_human-gene(s),RNA_human-breakpoint,RNA_human-gene(s)
32,HTMCP-03-06-02204,HPV_18,reviewed_confirmed_hpv18_integration,integrated,NO,na,na,19_42217693,-


,patient,virus,viral-breakpoint_DNA,viral-gene_DNA,DNA_lib_DNA,RNA_lib_DNA,viral-breakpoint_RNA,viral-gene_RNA,DNA_lib_RNA,RNA_lib_RNA
98,HTMCP-03-06-02217,HPV_16,NaN,NaN,NaN,NaN,227;881;881;1319;3432;3452,NA,A57088,A57115
99,HTMCP-03-06-02217,HPV_18,NaN,NaN,NaN,NaN,929,NA,A57088,A57115


,patient,virus,comments,integration_status,should_exclude_due_to_review,DNA_human-breakpoint,DNA_human-gene(s),RNA_human-breakpoint,RNA_human-gene(s)


In [351]:
dfrm[dfrm.patient == 'HTMCP-03-06-02217']

,library,virus,Integration,human-breakpoint,human-gene(s),viral-breakpoint,viral-gene,Tissue Site,HIV status,GSC original source ID,patient,DNA_lib,status,RNA_lib
504,A57115,HPV_16,YES,4_8848052,intergenic,227,NA,Cervix,Positive,HTMCP_403,HTMCP-03-06-02217,A57088,Malignant,A57115
505,A57115,HPV_16,YES,4_8848057,intergenic,881,NA,Cervix,Positive,HTMCP_403,HTMCP-03-06-02217,A57088,Malignant,A57115
506,A57115,HPV_16,YES,4_8848715,intergenic,881,NA,Cervix,Positive,HTMCP_403,HTMCP-03-06-02217,A57088,Malignant,A57115
507,A57115,HPV_16,YES,4_8849923,intergenic,1319,NA,Cervix,Positive,HTMCP_403,HTMCP-03-06-02217,A57088,Malignant,A57115
508,A57115,HPV_16,YES,4_8849938,intergenic,3432,NA,Cervix,Positive,HTMCP_403,HTMCP-03-06-02217,A57088,Malignant,A57115
509,A57115,HPV_16,YES,4_8850618,intergenic,3452,NA,Cervix,Positive,HTMCP_403,HTMCP-03-06-02217,A57088,Malignant,A57115
510,A57115,HPV_18,YES,19_42218888,CEACAM5,929,NA,Cervix,Positive,HTMCP_403,HTMCP-03-06-02217,A57088,Malignant,A57115
511,A57115,"Alphapapillomavirus 9 isolate IARC1100085,",NO,-,-,-,-,Cervix,Positive,HTMCP_403,HTMCP-03-06-02217,A57088,Malignant,A57115
512,A57115,"Alphapapillomavirus 9 isolate PAP0966,",NO,-,-,-,-,Cervix,Positive,HTMCP_403,HTMCP-03-06-02217,A57088,Malignant,A57115


In [352]:
df5[df5.patient == 'HTMCP-03-06-02217']

,patient,virus,comments,integration_status,should_exclude_due_to_review,DNA_human-breakpoint,DNA_human-gene(s),RNA_human-breakpoint,RNA_human-gene(s)
23,HTMCP-03-06-02217,HPV_16,E6_7_expressed,unintegrated,NO,na,na,-,-


# meta data

In [151]:
f = '/home/szong/projects/meta/META_2019/Master_Sample_table_14X292_20190910_SZ.xls'
df = pd.read_excel(f, sheet_name='HIV Cervical Genome', 
                   usecols=['Biospecimen ID', 'Tissue Status', 'Tissue Site', 
                            'HIV status', 'DNA Library ID', 'GSC original source ID'])
# both works
# df['Biospecimen ID'] = df['Biospecimen ID'].str.split('-').apply(lambda x: '-'.join(x[:4]))
df['Biospecimen ID'] = df['Biospecimen ID'].str.split('-').str[:4].str.join('-')
df.head(2)

,Biospecimen ID,Tissue Status,Tissue Site,HIV status,DNA Library ID,GSC original source ID
0,HTMCP-03-06-02001,Malignant,Cervix,positive,A37234,HTMCP_124
1,HTMCP-03-06-02001,Normal,Peripheral blood,positive,A37159,HTMCP_176


#### 118 patients

In [152]:
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/118_patients.txt'
pats = pd.read_csv(f1, header=None)[0]
pats[:3]

0    HTMCP-03-06-02001
1    HTMCP-03-06-02002
2    HTMCP-03-06-02003
Name: 0, dtype: object

#### top ups

In [153]:
df[df['Biospecimen ID']=='HTMCP-03-06-02092']
df.drop(61, inplace=True)
df = df[(df['Biospecimen ID'].isin(pats))&(df['Tissue Status']!='Normal')]
assert df.shape[0] == 118

,Biospecimen ID,Tissue Status,Tissue Site,HIV status,DNA Library ID,GSC original source ID
61,HTMCP-03-06-02092,Malignant,Cervix,Negative,A37282,HTMCP_172
62,HTMCP-03-06-02092,Malignant,Cervix,Negative,A60755,HTMCP_172
63,HTMCP-03-06-02092,Normal,Peripheral blood,Negative,A37210,HTMCP_224


#### RNA lib ids

In [154]:
f3 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/124_patients_bam_vcf_20190115.csv'
df3 =  pd.read_csv(f3, usecols=['patient', 'DNA_lib', 'RNA_lib', 'status'])
df3 = df3[df3.status=='Malignant']
df3.head(2)
df3.shape

,patient,DNA_lib,status,RNA_lib
0,HTMCP-03-06-02001,A37234,Malignant,A37700
1,HTMCP-03-06-02002,A37235,Malignant,A37701


(124, 4)

In [155]:
df4 = pd.merge(df, df3, left_on='Biospecimen ID', right_on='patient', how='left')
df4.drop(['Biospecimen ID', 'DNA Library ID', 'Tissue Status'], axis=1, inplace=True)
df4.head(2)

,Tissue Site,HIV status,GSC original source ID,patient,DNA_lib,status,RNA_lib
0,Cervix,positive,HTMCP_124,HTMCP-03-06-02001,A37234,Malignant,A37700
1,Cervix,Negative,HTMCP_125,HTMCP-03-06-02002,A37235,Malignant,A37701


In [156]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/genomes_with_htmcp_ids_118_20191217.tsv'
df4.to_csv(of, sep='\t', index=False)